In [ ]:
# ⬇️ CLIQUE NA SETA PARA INSTALAR (demora 2-3 minutos na primeira vez)

%pip install polars requests plotly

print("✅ PRONTO! Agora vá para o próximo código")


In [ ]:
# ⬇️ CLIQUE AQUI PARA BAIXAR OS DADOS (demora 3-5 minutos)

import polars as pl
import requests
import time

def baixar_dados_drive():
    """Baixa automaticamente o arquivo de grandes litigantes"""
    file_id = "1Ns07hTZaK4Ry6bFEHvLACZ5tHJ7b-C2E"
    nome_arquivo = "grandes_litigantes_202504.parquet"
    
    print("📥 Baixando dados dos grandes litigantes...")
    print("⏳ Aguarde: isso pode demorar alguns minutos...")
    
    # Múltiplas URLs para garantir download
    urls = [
        f"https://drive.google.com/uc?export=download&id={file_id}",
        f"https://drive.google.com/uc?id={file_id}&export=download"
    ]
    
    for i, url in enumerate(urls):
        try:
            print(f"🔄 Tentativa {i+1}: {url[:50]}...")
            
            session = requests.Session()
            response = session.get(url, stream=True)
            
            # Verificar confirmação de vírus do Google
            if 'confirm=' in response.text and 'download' not in response.url:
                print("🔒 Confirmando download no Google Drive...")
                confirm_url = f"https://drive.google.com/uc?export=download&confirm=t&id={file_id}"
                response = session.get(confirm_url, stream=True)
            
            if response.status_code == 200 and len(response.content) > 1000000:  # > 1MB
                print("💾 Salvando arquivo...")
                with open(nome_arquivo, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                
                # Verificar se foi baixado corretamente
                try:
                    test_df = pl.scan_parquet(nome_arquivo)
                    rows = test_df.select(pl.len()).collect().item()
                    print(f"✅ SUCESSO! Arquivo baixado: {rows:,} registros")
                    return nome_arquivo
                except Exception as e:
                    print(f"⚠️ Arquivo corrompido: {e}")
                    continue
            else:
                print(f"❌ Status {response.status_code} ou arquivo muito pequeno")
                
        except Exception as e:
            print(f"❌ Erro na tentativa {i+1}: {str(e)[:100]}")
            
    print("❌ Não foi possível baixar. Tente novamente!")
    return None

# EXECUTAR DOWNLOAD
arquivo = baixar_dados_drive()

if arquivo:
    print("\n🎉 DADOS PRONTOS! Vá para o próximo código")
else:
    print("\n⚠️ PROBLEMA no download. Execute este código novamente!")
